In [7]:
from cmatcher.owl_utils import *
from cmatcher.cqa_search import *
from cmatcher.eval_utils import *
from transformers import AutoTokenizer
import dill
import os
import torch.optim as optim
import copy
from accelerate import notebook_launcher

from cmatcher.model import *

from accelerate import Accelerator
import random
import wandb

import argparse

In [25]:
def parse_arguments():
    arg_parser = argparse.ArgumentParser(description='')

    arg_parser.add_argument('--sweep', dest='sweep', nargs='?', type=int)

    return arg_parser.parse_args()


os.environ["TOKENIZERS_PARALLELISM"] = 'false'

torch.manual_seed(0)
random.seed(0)

# args = parse_arguments()

test_onts = ['cmt', 'conference', 'confOf', 'edas', 'ekaw']
language_models = ['BAAI/bge-base-en', 'infgrad/stella-base-en-v2', 'BAAI/bge-large-en-v1.5', 'llmrails/ember-v1',
                   'thenlper/gte-large']
architectures = ['lm', 'gnn', 'sgnn']
lm_grad = ['none', 'grad']
pred = ['none', 'pred']
dephs = [1, 2, 3, 4]

In [26]:
def all_combinations():
    combs = []
    for to in test_onts:
        for lm in language_models:
            for a in architectures:
                if a == 'lm':
                    combs.append((to, lm, a, 'grad', 'none', 0))
                    continue
                for g in lm_grad:
                    for p in pred:
                        for d in dephs:
                            combs.append((to, lm, a, g, p, d))

    return combs


print(len(all_combinations()))

test_ont, language_model, architecture, grad, cpred, depth = all_combinations()[2]

825


ValueError: too many values to unpack (expected 5)

In [11]:
config = {
    'test_ont': test_ont,
    'learning_rate': 0.00001,
    'language_model': language_model,
    'architecture': architecture,
    'pred': cpred,
    'epochs': 5,
    'batch_size': 2,
    'evm_th': 0.9,
    'ev_sim_threshold': 0.8,
    'sim_margin': 0.8,
    'depth': depth,
    'grad': grad
}

wandb.init(
    project='cmatcher',
    config=config,
    group=f'{language_model}-{architecture}-{cpred}-{grad}',
    settings=wandb.Settings(_disable_stats=True, _disable_meta=True)
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: guihss-cs (ghss). Use `wandb login --relogin` to force relogin


In [12]:
ontology_paths = {
    'edas.owl': '/projets/melodi/gsantoss/data/oaei/tracks/conference/onts/edas.owl',
    'ekaw.owl': '/projets/melodi/gsantoss/data/oaei/tracks/conference/onts/ekaw.owl',
    'confOf.owl': '/projets/melodi/gsantoss/data/oaei/tracks/conference/onts/confOf.owl',
    'conference.owl': '/projets/melodi/gsantoss/data/oaei/tracks/conference/onts/conference.owl',
    'cmt.owl': '/projets/melodi/gsantoss/data/oaei/tracks/conference/onts/cmt.owl',
}

cqa_path = '/projets/melodi/gsantoss/data/complex/CQAs'
entities_path = '/projets/melodi/gsantoss/data/complex/entities-cqas'

if os.path.exists('/projets/melodi/gsantoss/tmp/idata.pkl'):
    with open('/projets/melodi/gsantoss/tmp/idata.pkl', 'rb') as f:
        train_ont_cqa_subg = dill.load(f)
        print('loaded from cache.')
else:
    with open('/projets/melodi/gsantoss/tmp/idata.pkl', 'wb') as f:
        dill.dump(load_entities(entities_path, ontology_paths), f)

isg = load_sg(entities_path, ontology_paths)

cqas = load_cqas(cqa_path)
raw_data = build_raw_data(train_ont_cqa_subg, cqas)

test_ont = config['test_ont']

if os.path.exists(f'/projets/melodi/gsantoss/tmp/{test_ont}.pkl'):
    with open(f'/projets/melodi/gsantoss/tmp/{test_ont}.pkl', 'rb') as f:
        ifd, mc, mp, fres = dill.load(f)
        print('loaded from cache.')
else:
    ifd, mc, mp, fres = build_raw_ts(f'/projets/melodi/gsantoss/data/oaei/tracks/conference/onts/{test_ont}.owl',
                                     isg[test_ont],
                                     workers=4)
    with open(f'/projets/melodi/gsantoss/tmp/{test_ont}.pkl', 'wb') as f:
        dill.dump((ifd, mc, mp, fres), f)

conts_cqa_subg = copy.deepcopy(train_ont_cqa_subg)
del conts_cqa_subg[test_ont]

tokenizer = AutoTokenizer.from_pretrained(config['language_model'])

root_entities, graph_data, cq, cqid, caq, cqmask, tor = prepare_eval_dataset(test_ont, cqas, ifd, tokenizer, mc, mp,
                                                                             fres)


loaded from cache.


  0%|          | 0/101 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

loaded from cache.


In [14]:
def train_function(config, model, root_entities, graph_data, cq, cqid, res, caq, cqmask, tor):
    accelerator = Accelerator()
    optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])

    lh = []
    evh = []
    epochs = config['epochs']
    batch_size = config['batch_size']
    progress = None

    triplet_loss = nn.TripletMarginWithDistanceLoss(
        distance_function=lambda x, y: 1.0 - torch.cosine_similarity(x, y), margin=config['sim_margin'])

    dataset = CQADataset(tokenizer, conts_cqa_subg, raw_data[test_ont], filter_bn=False)
    loader = DataLoader(dataset, batch_size=batch_size)

    cqloader = DataLoader(cqid, batch_size=batch_size, shuffle=False)
    acqloader = [DataLoader(a, batch_size=batch_size, shuffle=False) for a in caq]
    graph_loader = DataLoader(graph_data, batch_size=batch_size, shuffle=False)

    model, optimizer, loader, cqloader, graph_loader, *acqloader = accelerator.prepare(model, optimizer, loader,
                                                                                       cqloader, graph_loader,
                                                                                       *acqloader)
    model.find_unused_parameters = False
    if not progress and accelerator.is_main_process:
        progress = tqdm(total=epochs * len(loader))

    accelerator.print('First evaluation:')
    eval_test(accelerator, model, cqloader, graph_loader, cq, root_entities, res, acqloader, cqmask, tor)

    accelerator.print(f'ev data: {evm(accelerator, model, dataset, th=config["ev_sim_threshold"]):.2f}')

    for e in range(epochs):

        model.train()

        el = []
        for batch in loader:
            with accelerator.accumulate(model):
                optimizer.zero_grad()

                cqs, sbgs, nsbg = model(cqa=batch.cqs, positive_sbg=(batch.x_sf, batch.x_s,
                                                                     batch.edge_index_s, batch.edge_feat_sf,
                                                                     batch.edge_feat_s),
                                        negative_sbg=(batch.x_nf, batch.x_n,
                                                      batch.edge_index_n, batch.edge_feat_nf, batch.edge_feat_n))

                isbgs = sbgs[batch.rsi]
                isbgn = nsbg[batch.rni]

                loss = triplet_loss(cqs, isbgs, isbgn)
                el.append(accelerator.gather_for_metrics(loss.detach()))
                accelerator.backward(loss)

                optimizer.step()
                if accelerator.is_main_process:
                    progress.update(1)

        lh.append(torch.stack(el).mean().item())

        evh.append(evm(accelerator, model, dataset, th=config["ev_sim_threshold"]))
        eval_test(accelerator, model, cqloader, graph_loader, cq, root_entities, res, acqloader, cqmask, tor)
        accelerator.print(f'epoch {e} loss: {lh[-1]:.2f}, ev: {evh[-1]:.2f}')
        wandb.log({'global/acc': evh[-1], 'global/loss': lh[-1]})

    if accelerator.is_main_process:
        progress.close()


model = Model(config['language_model'], d=config['depth'], lm_grad=config['grad'] == 'grad')
notebook_launcher(train_function, (config, model, root_entities, graph_data, cq, cqid, fres, caq, cqmask, tor), num_processes=2)
wandb.finish()

Launching training on 2 GPUs.


Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


  0%|          | 0/520 [00:00<?, ?it/s]

First evaluation:
bt: 0.80 avgp: 0.06, rec: 0.65, afm: 0.11
edas bt: 0.80 avgp: 0.79, rec: 0.25, afm: 0.38
confOf bt: 0.80 avgp: 0.86, rec: 0.11, afm: 0.20
ekaw bt: 0.80 avgp: 0.57, rec: 0.10, afm: 0.17
conference bt: 0.80 avgp: 0.46, rec: 0.14, afm: 0.22
ev data: 0.29
bt: 0.80 avgp: 0.83, rec: 0.88, afm: 0.85
edas bt: 0.60 avgp: 0.50, rec: 0.62, afm: 0.56
confOf bt: 0.45 avgp: 0.34, rec: 0.56, afm: 0.42
ekaw bt: 0.60 avgp: 0.39, rec: 0.50, afm: 0.44
conference bt: 0.60 avgp: 0.34, rec: 0.36, afm: 0.35
epoch 0 loss: 0.59, ev: 0.98
bt: 0.90 avgp: 0.88, rec: 0.88, afm: 0.88
edas bt: 0.45 avgp: 0.42, rec: 0.62, afm: 0.50
confOf bt: 0.30 avgp: 0.30, rec: 0.67, afm: 0.41
ekaw bt: 0.45 avgp: 0.30, rec: 0.50, afm: 0.38
conference bt: 0.45 avgp: 0.29, rec: 0.36, afm: 0.32
epoch 1 loss: 0.34, ev: 0.99
bt: 0.85 avgp: 0.88, rec: 0.88, afm: 0.88
edas bt: 0.60 avgp: 0.63, rec: 0.62, afm: 0.63
confOf bt: 0.35 avgp: 0.33, rec: 0.67, afm: 0.45
ekaw bt: 0.55 avgp: 0.28, rec: 0.50, afm: 0.36
conference 

wandb: WARNING Ensure read and write access to run files dir: /users/melodi/gsantoss/wandb/run-20240206_104309-jjuh6qys/files, control this via the WANDB_DIR env var. See https://docs.wandb.ai/guides/track/environment-variables


each/confOf-afm,▁▅▅▆▆█
each/confOf-avgp,█▁▁▁▃▃
each/confOf-bt,█▃▁▂▆▂
each/confOf-rec,▁▇██▅█
each/conference-afm,▁▇▆▅█▆
each/conference-avgp,█▄▂▁▅▅
each/conference-bt,█▅▂▁▂▃
each/conference-rec,▁████▆
each/edas-afm,▁▅▄▇▇█
each/edas-avgp,█▃▁▅▂▇
each/edas-bt,█▄▁▄▁█
